Import libraries

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import os
import geopandas as gpd
import copy as cp

In [2]:
# get data and write to files

ic_url = 'http://opendatacommunities.org/downloads/cube-table?uri=http%3A%2F%2Fopendatacommunities.org%2Fdata%2Fsocietal-wellbeing%2Fdeprivation%2Fimd-income-score-2010'

r = requests.get(ic_url)
name = ic_url.split('%2F')[-2] + '_' + ic_url.split('%2F')[-1]
fo = open(name + '.csv', 'wb')
fo.write(r.content)
fo.close()

In [15]:
# load files (those that were just downloaded, plus the census one)

imd_income = pd.read_csv('deprivation_imd-income-score-2010.csv')
cen_nssec = pd.read_csv('Data_Health_Job.csv')
cen_health = pd.read_csv('Data_Heating.csv')
cen_heat = pd.read_csv('Data_Disability.csv')
cen_depr = pd.read_csv('Data_Deprivation.csv')
cen_socec = pd.read_csv('Data_SocEc.csv')

In [4]:
# make functions to organise data

def get_meta_data(data):
    # remove unwanted columns
    meta_data = (data.drop(['CDU_ID', 'GEO_CODE', 'GEO_LABEL', 'GEO_TYPE', 'GEO_TYP2'], axis = 1)\
                .iloc[:1]\
                .T
                )
    
    # separate into various columns
    for i in range(meta_data.iloc[0][0].count(' - ')):
        name = str(meta_data.iloc[0][0].split(' - ')[i].split(' : ')[0])
        
        # extract individual strings
        temp_list = []
        for j in range(len(meta_data)):
            var_string = str(meta_data.iloc[j][0].split(' - ')[i].split(' : ')[1])
            temp_list.append(var_string)
            #print(temp_list)

        # make columns
        meta_data[name] = temp_list
        #print(temp_list)
        
    # remove unwanted column
    meta_data = meta_data.drop([0], axis = 1)
    
    # clean headers
    meta_data.columns = meta_data.columns.str.strip().str.lower().str.replace(' ', '_').str.replace(')', '').str.replace('(', '').str.replace(';', '')
        
    return(meta_data)


def clean_census_data(data):
    # remove unwanted columns
    clean_data = (data.drop(['CDU_ID', 'GEO_LABEL', 'GEO_TYPE', 'GEO_TYP2'], axis =1)\
                  .drop([0], axis = 0)\
                  .set_index('GEO_CODE')\
                  .T)
    
    # merge with census data
    #meta_data = get_meta_data(data)
    #clean_data = meta_data.join(clean_data)
    
    return(clean_data)

def filter_totals(cen_data):
    
    meta_data = get_meta_data(cen_data)

    for i in range(len(meta_data)):
        for j in range(len(meta_data.columns)):
            if 'Total\ ' in meta_data.iloc[i][j]: #dat.iloc[1][3] # [row] [column]
                meta_data.iloc[i][j] = np.nan
            
    meta_data = meta_data.dropna(axis = 0)
    
    # get column names for filtering
    cl_cen_dat = clean_census_data(cen_data)
    col_list = []
    for i in range(len(cl_cen_dat.columns)):
        if 'E01' in cl_cen_dat.columns[i]:
            col_list.append(cl_cen_dat.columns[i])
    
    # merge with census data
    data = meta_data.join(cl_cen_dat)
    data = data[col_list]
    
    return(data.T)

In [5]:
def pnt_of_tot(old_data):
    
    data = old_data.T.astype(int)
    data['sum'] = data.sum(axis = 1, skipna = all)
    
    new_df = pd.DataFrame(index = data.index, columns = data.columns)
    for i in range(len(data)):
        for j in range(int(len(data.columns) - 1)):
            new_df.iloc[i][j] = (data.iloc[i][j] / data.iloc[i][int(len(data.columns) - 1)] * 100)
    
    new_df = new_df.dropna(axis = 1)

    return(new_df)

In [23]:
# clean dataframes

#dat_nssec = pnt_of_tot(filter_totals(cen_nssec)) # done!
#dat_health = pnt_of_tot(filter_totals(cen_health)) # empty?
#dat_depr = pnt_of_tot(filter_totals(cen_depr)) # done!
dat_socec = pnt_of_tot(filter_totals(cen_socec))
dat_heat = pnt_of_tot(clean_census_data(cen_heat).T) # have to use clean_census_data() here becasue I only downloaded totals

KeyboardInterrupt: 

In [8]:
dat_nssec = pnt_of_tot(filter_totals(cen_nssec))

In [19]:
dat_socec.T.to_csv('cleaned_socec.csv')

In [21]:
dat_socec.head()

,E01004766,E01004767,E01004768,E01004769,E01004770,E01004771,E01004772,E01004773,E01004774,E01004775,...,E01033680,E01033681,E01033682,E01033683,E01033684,E01033685,E01033686,E01033687,E01033688,sum


In [7]:
cen_nssec.head()

,CDU_ID,GEO_CODE,GEO_LABEL,GEO_TYPE,GEO_TYP2,F46769,F46773,F46783,F46787,F46791,...,F39439,F39441,F39442,F39443,F39445,F39446,F39447,F39449,F39450,F39451
0,NaN,NaN,NaN,NaN,NaN,Age : Age 50 and over - Long-term health probl...,Age : Age 50 and over - Long-term health probl...,Age : Age 50 and over - Long-term health probl...,Age : Age 50 and over - Long-term health probl...,Age : Age 50 and over - Long-term health probl...,...,Age : Age 35 to 49 - General health : Fair hea...,Age : Age 35 to 49 - General health : Fair hea...,Age : Age 35 to 49 - General health : Fair hea...,Age : Age 35 to 49 - General health : Fair hea...,Age : Age 35 to 49 - General health : Fair hea...,Age : Age 35 to 49 - General health : Fair hea...,Age : Age 35 to 49 - General health : Fair hea...,Age : Age 35 to 49 - General health : Fair hea...,Age : Age 35 to 49 - General health : Fair hea...,Age : Age 35 to 49 - General health : Fair hea...
1,23905.0,E01004766,Bolton 005A,Lower Super Output Areas and Data Zones,LSOADZ,613,30,109,61,66,...,13,5,3,2,14,8,6,17,12,5
2,23906.0,E01004767,Bolton 005B,Lower Super Output Areas and Data Zones,LSOADZ,571,36,108,72,63,...,25,5,2,3,16,6,10,23,11,12
3,23907.0,E01004768,Bolton 001A,Lower Super Output Areas and Data Zones,LSOADZ,581,105,171,97,84,...,12,3,1,2,8,3,5,10,5,5
4,23908.0,E01004769,Bolton 003A,Lower Super Output Areas and Data Zones,LSOADZ,592,81,157,113,55,...,12,4,2,2,6,2,4,14,8,6


In [ ]:
neis = gpd.read_file('shapefiles/Unit2_exercise1Data/with_OA.shp')

In [ ]:
neis.plot()

In [ ]:
neis = gpd.read_file('lsoas/Lower_Layer_Super_Output_Areas_December_2001_Generalised_Clipped_Boundaries_in_England_and_Wales.shp')

In [ ]:
neis.plot()